# **Main**

In [0]:
import csv
import re
import os
from requests import get
import requests                
from bs4 import BeautifulSoup

In [0]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

import numpy as np
from collections import Counter

def EPE(s):
  """correct punctuation"""
  s = s.strip()
  if len(s) > 1:
      frequence = Counter(s)
      frequence = [frequence["?"], frequence["!"], frequence["."]] 
      end = '.' if max(frequence) == 0 else ["?", "!","."][np.argmax(frequence)]
      """Traite les espacement inutiles et enleve les points"""
      s = s.replace('  ', ' ').replace('  ', ' ')
      s = s.replace(' . ', ' ').replace('. ', ' ').replace(' .', ' ').replace('.', ' ')
      s = s.replace(' ! ', ' ').replace('! ', ' ').replace(' !', ' ').replace('!', ' ')
      s = s.replace(' ? ', ' ').replace('? ', ' ').replace(' ?', ' ').replace('?', ' ')
      try :
        if s[-1] in [',',';']:
          s = s[:len(s)-1]
      except Exception as e :
        pass
      s = s.replace('  ', ' ').replace('  ', ' ').strip() +''+ end
  return s

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
class ScrapperYv :
  def __init__(self, main_link, prefix_at, prefix_nt, YourVersion_at, YourVersion_nt, prefix_sufix_reference):
    self.main_link = main_link
    self.prefix_at = prefix_at
    self.prefix_nt = prefix_nt
    self.YourVersion_at = YourVersion_at
    self.YourVersion_nt = YourVersion_nt
    self.prefix_sufix_reference = prefix_sufix_reference
  
  def meta_data(self, main_path, txt_separator = '__SEPARATOR__', cell_error= '__Error__', into_link = 'INTRO1'):
    self.main_path = main_path
    self.txt_separator = txt_separator 
    self.cell_error = cell_error
    self.into_link = into_link

  def getVesetOfChapter(self, chapterLink, chapterPrefix):
    print(chapterLink) ###
    print("chapterPrefix :",chapterPrefix)
    data = {}
    response = requests.get(chapterLink) 
    div_chapter = BeautifulSoup(
        requests.get(chapterLink).text, 
        "html.parser"
    ).select("div[class^='chapter ch']")[0]
    for div in div_chapter.findAll("div") :
      for v_span in [v_span for v_span in div.select("span[class^='verse v']") if not(len(v_span.text) == 1)] :
        versets_number = v_span["data-usfm"]

        #print("versets_number :",versets_number)
        #versets_number_temoin = chapterPrefix + versets_number.split(".")[2]
        #print("versets_number_temoin : ", versets_number_temoin)
        
        contents_span = v_span.findAll("span", {"class": "content"})
        versets_content = contents_span[0].text
        del contents_span[0]
        for content_span in contents_span :
          versets_content = versets_content + ' ' + content_span.text
        try :
          data[versets_number] 
          data[versets_number] = data[versets_number] + ' ' + versets_content
        except :
          data[versets_number] = versets_content
    k = 1
    for h_span in [h_span for h_span in div_chapter.findAll("span", {"class": "heading"}) if not(len(h_span.text.replace(' ','')) == 0)] :
        data[chapterPrefix+"heading."+str(k)] = h_span.text
        k = k + 1
    return data    

  def WriteIntroduction(self, prefix_livre, langue):
    contents = BeautifulSoup(
      requests.get(
        self.main_link + langue[1] + prefix_livre + self.into_link + langue[2]
      ).text, 
      "html.parser"
    ).findAll("span", {"class": "content"})
    try :
      contents[0]
      repertoire = self.main_path+"introductions/"+langue[0]
      if not os.path.exists(repertoire):
        os.makedirs(repertoire)
      with open(repertoire+"/"+prefix_livre+'txt', "w", encoding="ISO-8859-1") as fichier:
        for c in contents :
          fichier.writelines(c.text)
    except :
      pass
      
  def getNombreVersetOfChapter(self, prefix_livre, chapter):
    return self.getVesetOfChapter(
            chapterLink=(self.main_link +self.prefix_sufix_reference[0]+prefix_livre+str(chapter) + self.prefix_sufix_reference[1]), 
            chapterPrefix=(prefix_livre+str(chapter)+'.')
        )

  def writeError(self, errors_dic):
    for type_error, errors_dic_value in errors_dic.items():
      for langue, langue_dic_error in errors_dic_value.items() :
        repertoire = self.main_path+"errors/"+type_error+"/"+langue
        if not os.path.exists(repertoire):
          os.makedirs(repertoire)
        for prefix_livre_chap, versets in langue_dic_error.items() :
          with open(repertoire+"/"+ prefix_livre_chap +"."+'csv', 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=['livre.chapitre.verset', 'verset'])
            writer.writeheader()
            for livre_chapitre_verset, verset in versets.items():
              writer.writerow({
                  'livre.chapitre.verset': livre_chapitre_verset, 
                  'verset': verset
              })

  def correctError(self, link=None, index_ref=None):
    if (link and index_ref) :
      # todo : chercher la cause de l'erreur et corriger afin de retourner le bon résultat
      return self.cell_error 
    else :
      return self.cell_error 
 
  def scrap(self, YourVersion_dic, prefix_dic):
    errors = {}
    errors['overflow'] = {}
    errors['underflow'] = {}
    for prefix_livre, nb_chapitre in YourVersion_dic.items():
      with open(self.main_path+"csvs/"+prefix_livre+'csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = ['livre.chapitre.verset']+list(prefix_dic.keys()))
        writer.writeheader()
        data = {} 
        data_zip = {}
        for i in range(1,nb_chapitre+1):
          index_ref = list(self.getNombreVersetOfChapter(prefix_livre=prefix_livre, chapter=i).keys())
          nb_verset  = len(index_ref)
          for langue, (prefix, suffix) in prefix_dic.items() :
            self.WriteIntroduction(prefix_livre=prefix_livre, langue=[langue, prefix, suffix]) # Introduction
            print(langue, prefix, suffix) ###
            chapterLink=self.main_link + prefix + prefix_livre + str(i) + suffix
            versets = self.getVesetOfChapter(chapterLink=chapterLink, chapterPrefix=(prefix_livre+str(i)+'.'))
            print(versets) ###
            l = len(versets)
            if(l > nb_verset):
              try :
                errors['overflow'][langue][prefix_livre+str(i)] = versets
              except :
                errors['overflow'][langue] = {}
                errors['overflow'][langue][prefix_livre+str(i)] = versets
              # On diminue
              a = {}
              for key in index_ref :
                try :
                  a[key] = versets[key]
                except :
                  a[key] = self.correctError(link=chapterLink, index_ref=index_ref)
              verserts = a
              #versets = dict(list(versets.items())[:nb_verset]) 
            elif (l < nb_verset):
              try :
                errors['underflow'][langue][prefix_livre+str(i)] = versets 
              except :
                errors['underflow'][langue] = {}
                errors['underflow'][langue][prefix_livre+str(i)] = versets 
              # On complete
              print(list(versets.keys())) ###
              for key in index_ref :
                try :
                  versets[key]
                except :
                  versets[key] = self.correctError(link=chapterLink, index_ref=index_ref)
              print(list(versets.keys())) ###
              #versets.update(dict((prefix_livre+str(i)+'.'+str(j), '__Error__') for j in range(l, nb_verset+1))) 
            data_zip[langue]=versets
          #print(data_zip)
          index = list(data_zip[list(data_zip.keys())[0]].keys())
          print(index) ###
          for i in range(nb_verset):            
            data["livre.chapitre.verset"] = index[i]
            for langue in data_zip.keys() :
              try :
                data[langue] = data_zip[langue][index[i]]
              except :
                print(index)
                print(langue)
                print(data_zip[langue])
                data[langue] = self.correctError()
            writer.writerow(data)
    self.writeError(errors_dic=errors)
  
  def get_XLM_data(self, csv_path, output_dir, data_type = "mono", langues=[]):
    """
    """
  
    # Si aucune langue n'est spécifiée, on selectionne tout les langues
    if langues == [] :
      langues = list(self.prefix_nt.keys())
    
    fichiers = list(self.YourVersion_at.keys()) + list(self.YourVersion_nt.keys())

    l = len(langues)
    for i in range(l-1):
      for j in range(i+1, l):
        li = langues[i]
        lj = langues[j]
        print(li + '---' + lj)

        repertoire = output_dir+"/"+data_type
        if not os.path.exists(repertoire):
          os.makedirs(repertoire)

        if data_type == "para":
          repertoire = repertoire +"/"+ li +'-'+ lj
          if not os.path.exists(repertoire):
            os.makedirs(repertoire)
          repertoire = [repertoire + "/" + li + '-' + lj + "." for _ in range(2)]
        elif data_type == "mono":
          repertoire = [repertoire + "/" + li, repertoire +"/"+ lj]
          for rep in repertoire :
            if not os.path.exists(rep):
              os.makedirs(rep)
          repertoire = [r +"/" for r in repertoire]

        with open(repertoire[0] + li + '.txt', 'w') as txtfile1:
          with open(repertoire[1] + lj + '.txt', 'w') as txtfile2:
            
            for fichier in fichiers:
              try :
                with open(csv_path+"/csvs/"+fichier+'csv', 'r') as csvfile:
                  f_csv = csv.reader(csvfile)
                  filenames = []
                  try :
                    fieldnames = next(f_csv)
                  except StopIteration as si :
                    print(si)
                  try :
                    index_i = fieldnames.index(li)
                    index_j = fieldnames.index(lj)
                    versert = fieldnames.index("livre.chapitre.verset")
                    for ligne in f_csv:
                      x_i = ligne[index_i] 
                      y_i =  ligne[index_j]
                      if x_i != self.cell_error and y_i != self.cell_error :
                          txtfile1.writelines(ligne[versert]+" -- "+x_i+'\n')
                          txtfile2.writelines(ligne[versert]+" -- "+y_i+'\n')
                  except Exception as ex:
                    print(ex)
              except Exception as ex :
                print(ex)          
    
  def toTxtProfondeur(self, with_EPE=False, binaire=False):
    """couteux, efficace"""
    ref = 'p' + ('b' if binaire else 'nb')  +  ('2' if with_EPE else '1') 
    mode = 'wb' if binaire else 'w' 
    langues = list(self.prefix_nt.keys())
    fichiers = list(self.YourVersion_at.keys()) + list(self.YourVersion_nt.keys())
    l = len(langues)
    for i in range(l-1):
      for j in range(i+1, l):
        with open(self.main_path+"txts/"+langues[i]+'_'+langues[j]+'_'+ref+'.csv', mode, newline='') as csvfile1:
          with open(self.main_path+"txts/"+langues[j]+'_'+langues[i]+'_'+ref+'.csv', mode, newline='') as csvfile2:
            with open(self.main_path+"txts/"+langues[i]+'_'+langues[j]+'_'+ref+'.txt', mode) as txtfile1:
              with open(self.main_path+"txts/"+langues[j]+'_'+langues[i]+'_'+ref+'.txt', mode) as txtfile2:
                li = langues[i]
                lj = langues[j]
                print(li + '---' + lj)
                writer1 = csv.DictWriter(csvfile1, fieldnames = [li, lj])
                writer2 = csv.DictWriter(csvfile2, fieldnames = [lj, li])
                writer1.writeheader()
                writer2.writeheader()
                for fichier in fichiers:
                  with open(self.main_path+"csvs/"+fichier+'csv', 'r') as csvfile:
                    f_csv = csv.reader(csvfile)
                    fieldnames = next(f_csv)
                    print(fieldnames)
                    try :
                      index_i = fieldnames.index(li)
                      index_j = fieldnames.index(lj)
                      for ligne in f_csv:
                        if not with_EPE :
                          # csv
                          writer1.writerow({li : ligne[index_i], lj : ligne[index_j]})  
                          writer2.writerow({lj : ligne[index_j], li : ligne[index_i]})
                          # txt
                          txtfile1.writelines(ligne[index_i]+self.txt_separator+ligne[index_j]+'\n')
                          txtfile2.writelines(ligne[index_j]+self.txt_separator+ligne[index_i]+'\n')
                        else :
                          lignes_i = ligne[index_i].split('.')
                          lignes_j =ligne[index_j].split('.')
                          l = len(lignes_i)
                          if l == len(lignes_j) :
                            for k in range(l) :
                              # csv
                              writer1.writerow({li : EPE(lignes_i[k]), lj : EPE(lignes_j[k])})  
                              writer2.writerow({lj : EPE(lignes_j[k]), li : EPE(lignes_i[k])})
                              # txt
                              txtfile1.writelines(EPE(lignes_i[k])+self.txt_separator+EPE(lignes_j[k])+'\n')
                              txtfile2.writelines(EPE(lignes_j[k])+self.txt_separator+EPE(lignes_i[k])+'\n')
                          else :
                            lignes_i = EPE(ligne[index_i])
                            lignes_j = EPE(ligne[index_j])
                            # csv
                            writer1.writerow({li : lignes_i, lj : EPE(lignes_j[k])})  
                            writer2.writerow({lj : lignes_i, li : lignes_i})
                            # txt
                            txtfile1.writelines(lignes_i+self.txt_separator+lignes_j+'\n')
                            txtfile2.writelines(lignes_j+self.txt_separator+lignes_i+'\n')
                    except Exception as ex:
                      print(ex)

  def toTxtLargeur(self, with_EPE=False, binaire=False):
    """peux couteux, inefficace"""
    ref = 'l' + ('b' if binaire else 'nb')  +  ('2' if with_EPE else '1')
    fichiers = self.YourVersion_at
    fichiers.update(self.YourVersion_nt)
    for prefix_livre in fichiers.keys():
      with open(self.main_path+"csvs/"+prefix_livre+'csv', 'r') as csvfile:
        f_csv = csv.reader(csvfile)
        fieldnames = next(f_csv)
        l = len(fieldnames)
        # couplage deux à deux
        for i in range(1, l-1):
          for j in range(i+1, l):
            with open(self.main_path+"txts/"+fieldnames[i]+'_'+fieldnames[j]+'_'+ref+'.csv', 'a', newline='') as csvfile1:
              with open(self.main_path+"txts/"+fieldnames[j]+'_'+fieldnames[i]+'_'+ref+'.csv', 'a', newline='') as csvfile2:
                with open(self.main_path+"txts/"+fieldnames[i]+'_'+fieldnames[j]+'_'+ref+'.txt', 'a', encoding="UTF-8") as txtfile1:
                  with open(self.main_path+"txts/"+fieldnames[j]+'_'+fieldnames[i]+'_'+ref+'.txt', 'a', encoding="UTF-8") as txtfile2:
                    li = fieldnames[i]
                    lj = fieldnames[j]
                    writer1 = csv.DictWriter(txtfile1, fieldnames = [li, lj])
                    writer2 = csv.DictWriter(txtfile2, fieldnames = [lj, li])
                    writer1.writeheader()
                    writer2.writeheader()
                    if not with_EPE :
                      for ligne in f_csv:
                        # csv
                        writer1.writerow({li : ligne[i], lj : ligne[j]})  
                        writer2.writerow({lj : ligne[j], li : ligne[i]})
                        # txt
                        txtfile1.writelines(ligne[i]+self.txt_separator+ligne[j]+'\n')
                        txtfile2.writelines(ligne[j]+self.txt_separator+ligne[i]+'\n')
                    else :
                      lignes_i = ligne[i].split('.')
                      lignes_j = ligne[j].split('.')
                      l = len(lignes_i)
                      if l == len(lignes_j) :
                        for k in range(l) :
                          # csv
                          writer1.writerow({li : EPE(lignes_i[k]), lj : EPE(lignes_j[k])})  
                          writer2.writerow({lj : EPE(lignes_j[k]), li : EPE(lignes_i[k])})
                          # txt
                          txtfile1.writelines(EPE(lignes_i[k])+self.txt_separator+EPE(lignes_j[k])+'\n')
                          txtfile2.writelines(EPE(lignes_j[k])+self.txt_separator+EPE(lignes_i[k])+'\n')
                      else :
                        lignes_i = EPE(ligne[i])
                        lignes_j = EPE(ligne[j])
                        # csv
                        writer1.writerow({li : lignes_i, lj : lignes_j})  
                        writer2.writerow({lj : lignes_j, li : lignes_i})
                        # txt
                        txtfile1.writelines(lignes_i+self.txt_separator+lignes_j+'\n')
                        txtfile2.writelines(lignes_j+self.txt_separator+lignes_i+'\n')
             
  def run(self):
    # Ancien Testament
    #self.scrap(YourVersion_dic=self.YourVersion_at, prefix_dic=self.prefix_at)
    # Nouveau testament
    #self.scrap(YourVersion_dic=self.YourVersion_nt, prefix_dic=self.prefix_nt)
    # conversion
    self.toTxtProfondeur(with_EPE=False, binaire=False)
  
  def download(self):
    # Télécharger les fichiers en local
    try:
      from google.colab import files
    except ImportError:
      pass
    else:
      files.download('/content/drive/My Drive/datasets/YourVersion')

# **Géneral**

In [0]:
main_link = 'https://www.bible.com/fr/bible'
main_path = "/content/drive/My Drive/datasets/YourVersion/"

# langue : prefix, suffix
prefix_at = {'Francais':('/504/','.PDV2017'), 
             "Anglais":('/1/','.NIV'),  "BIBALDA_TA_PELDETTA":('/557/','.MASSANA'), 
             'Bulu':("/856/",".BULU09"),  'Guiziga':('/550/','.BEGDC'), "Fulfulde_Adamaoua":("/906/",'.FB'),  "Fulfulde_DC":('/905/','.FBDC'), 
              'KALATA_KO_SC_Gbaya':('/871/','.GB11'), 'KALATA_KO_DC_Gbaya':('/868/','.GB11DC'), 'Kapsiki_DC':('/1515/','.KBDC'),
              'Tupurri':('/892/','.TUPB')}
prefix_nt = {'Bafia':('/876/',".BAFNT"), 'Dii':('/1511/','.DIINT'), 'Ejagham':('/889/','.ETUNT'), 'Ghomala':('/907/','.GNT02'),  
             'Vute':('/887/','.NTV'), 'Limbum':('/1240/','.LNT'), 'MKPAMAN_AMVOE_Ewondo':('/1854/','.NTE12'), 'Mofa':('/908/',".MAFNT"), 
             "Mofu_Gudur":('/551/',".MOF"), "Ngiemboon":('/229/','.NBGM'), 'Doyayo':('/880/','.NTD'), "Guidar":('/1120/',".GDRNT85"),
            'Peere_Nt&Psalms':('/1237/','.PERE'), 'Samba_Leko':('/1118/','.SLNT'), "Du_na_sdik_na_wiini_Alaw":('/1855/',".MUSGUM")}
prefix_nt.update(prefix_at) # prefix_nt = {**prefix_at, **prefix_nt}

YourVersion_at = {
    'GEN.':50, 'EXO.':40, 'LEV.':27, 'NUM.':36, 'DEU.':34, 'JOS.':24, 
    'JDG.':21, 'RUT.':4,   '1SA.':31, '2SA.':24, '1KI.':22, '2KI.':25, 
    '1CH.':29, '2CH.':36, 'EZR.':10, 'NEH.':13, 'EST.':10, 'JOB.':42, 
    'PSA.':150, 'PRO.':31, 
    'ECC.':12, 'SNG.':8,  'ISA.':66, 'JER.':52,
    'LAM.':5, 'EZK.':48, 'DAN.':12, 'HOS.':14, 'JOL.':4, 'AMO.':9, 
    'OBA.':1, 'JON.':4, 'MIC.':7, 'NAM.':3, 'HAB.':3, 'ZEP.':3, 
    'HAG.':2, 'ZEC.':14, 'MAL.':3
}
YourVersion_nt = {
    'MAT.':28, 'MRK.':16, 'LUK.':24, 
    'JHN.':21, 'ACT.':28, 'ROM.':16, '1CO.':16, '2CO.':13, 'GAL.':6, 
    'EPH.':6, 'PHP.':4, 'COL.':4, '1TH.':5, '2TH.':3, '1TI.':6, 
    '2TI.':4, 'TIT.':3, 'PHM.':1, 'HEB.':13, 'JAS.':5, '1PE.':5, 
    '2PE.':3, '1JN.':5, '2JN.':1, '3JN.':1, 'JUD.':1, 'REV.':22
}

scrapperYv = ScrapperYv(main_link,prefix_at, prefix_nt, YourVersion_at, YourVersion_nt, prefix_sufix_reference=['/504/','.PDV2017'])
scrapperYv.meta_data(main_path = main_path, txt_separator="__SEPARATOR__", cell_error='__Error__', into_link="INTRO1")
scrapperYv.run()

In [0]:
path_to_file = "/content/drive/My Drive/datasets/YourVersion/txts/Francais_Bulu_pnb1.txt"
i = 0
with open(path_to_file, "r", encoding="ISO-8859-1") as fichier:
  for word in fichier.readlines() :
    i = i + 1
print(i)

29391


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
langues = list(prefix_nt.keys())
fichiers = list(YourVersion_at.keys()) + list(YourVersion_nt.keys())
l = len(langues)
for i in range(l-1):
  for j in range(i+1, l):
    try :
      print(os.remove("/content/drive/My Drive/datasets/YourVersion/txts/"+langues[i]+'_'+langues[j]+'_pnb2.csv')) 
    except :
      pass
    try :
      print(os.remove("/content/drive/My Drive/datasets/YourVersion/txts/"+langues[j]+'_'+langues[i]+'_pnb2.csv')) 
    except :
      pass
    try :
      os.remove("/content/drive/My Drive/datasets/YourVersion/txts/"+langues[i]+'_'+langues[j]+'_pnb2.txt') 
    except :
      pass
    try :
      os.remove("/content/drive/My Drive/datasets/YourVersion/txts/"+langues[j]+'_'+langues[i]+'_pnb2.txt') 
    except :
      pass

# **Les livres qui ont dérangés**


> RU, EST et DAN ont pris les données de GEN


> ACT est vide





In [0]:
main_link = 'https://www.bible.com/fr/bible'
main_path = "/content/"

In [0]:
! mkdir /content/csvs
#! rm -R /content/csvs

**RUTH : les suffixes changent pour les 03 premieres langues (mais le veritable probleme etait du au fait que j'avais ecrit RU au lieu de RUT).**

In [0]:
prefix_at = {
    'Francais':('/504/','.BCC1923'), "Anglais":('/1/','.KJV'),  "BIBALDA_TA_PELDETTA":('/557/','.Massana'), # 
    'Bulu':("/856/",".BULU09"),  'Guiziga':('/550/','.BEGDC'), "Fulfulde_Adamaoua":("/906/",'.FB'),  "Fulfulde_DC":('/905/','.FBDC'), 
     'KALATA_KO_SC_Gbaya':('/871/','.GB11'), 'KALATA_KO_DC_Gbaya':('/868/','.GB11DC'), 'Kapsiki_DC':('/1515/','.KBDC'),
      'Tupurri':('/892/','.TUPB')
  }

prefix_nt = {}
prefix_nt.update(prefix_at) 

YourVersion_at = {'RUT.':4}
YourVersion_nt = {}

#scrapperYv = ScrapperYv(main_link, prefix_at, prefix_nt, YourVersion_at, YourVersion_nt, prefix_sufix_reference=['/504/','.BCC1923'])
scrapperYv = ScrapperYv(main_link, prefix_at, prefix_nt, YourVersion_at, YourVersion_nt, prefix_sufix_reference=['/504/','.BCC1923'])
scrapperYv.meta_data(main_path = main_path, txt_separator="__SEPARATOR__", cell_error='__Error__', into_link="INTRO1")
scrapperYv.scrap(YourVersion_dic = scrapperYv.YourVersion_at, prefix_dic = scrapperYv.prefix_at)

https://www.bible.com/fr/bible/504/RUT.1.BCC1923
chapterPrefix : RUT.1.
Francais /504/ .BCC1923
https://www.bible.com/fr/bible/504/RUT.1.BCC1923
chapterPrefix : RUT.1.
{'RUT.1.1': " Au temps où les Juges gouvernaient, il y eut une famine dans le pays. Un homme de Bethléem de Juda s'en alla, avec sa femme et ses deux fils, séjourner dans les champs de Moab.", 'RUT.1.2': " Le nom de cet homme était Elimélech, le nom de sa femme était Noémi, et les noms de ses deux fils étaient Mahalon et Cheljon;  ils étaient  Ephratéens, de Bethléem de Juda. Ils allèrent aux champs de Moab, et s'y établirent.", 'RUT.1.3': ' Elimélech mari de Noémi, mourut, et elle resta seule avec ses deux fils,', 'RUT.1.4': " Il prirent des femmes moabites, dont l'une se nommait Orpha et l'autre se nommait Ruth, et ils demeurèrent là environ dix ans.", 'RUT.1.5': ' Mahalon et Cheljon moururent aussi tous deux, et la femme resta  privée  de ses deux fils et de son mari.', 'RUT.1.6': " Alors, s'étant levée, elle et ses b

In [0]:
! cp /content/csvs/RUT.csv /content/drive/"My Drive"/African_Translator/datasets/YourVersion/csvs

**EST et DAN : les deux premiers langues ont changé en tout**

In [0]:
prefix_at = {
    'Francais':('/2096/','.FRC97'), # https://my.bible.com/fr/bible/2096/EST.1.FRC97 et https://my.bible.com/fr/bible/2096/DAN.1.FRC97
    "Anglais":('/1588/','.AMP'),  # https://my.bible.com/fr/bible/1588/EST.4.AMP and https://my.bible.com/fr/bible/1588/DAN.11.AMP
    "BIBALDA_TA_PELDETTA":('/557/','.Massana'), 
    'Bulu':("/856/",".BULU09"),  'Guiziga':('/550/','.BEGDC'), "Fulfulde_Adamaoua":("/906/",'.FB'),  "Fulfulde_DC":('/905/','.FBDC'), 
     'KALATA_KO_SC_Gbaya':('/871/','.GB11'), 'KALATA_KO_DC_Gbaya':('/868/','.GB11DC'), 
     'Kapsiki_DC':('/1515/','.KBDC'), 'Tupurri':('/892/','.TUPB') 
  }

prefix_nt = {}
prefix_nt.update(prefix_at)

YourVersion_at = {'EST.':10, 'DAN.':12}
YourVersion_nt = {}

scrapperYv = ScrapperYv(main_link, prefix_at, prefix_nt, YourVersion_at, YourVersion_nt, prefix_sufix_reference=['/2096/','.FRC97'])
scrapperYv.meta_data(main_path = main_path, txt_separator="__SEPARATOR__", cell_error='__Error__', into_link="INTRO1")
scrapperYv.scrap(YourVersion_dic = scrapperYv.YourVersion_at, prefix_dic = scrapperYv.prefix_at)

https://www.bible.com/fr/bible/2096/EST.1.FRC97
chapterPrefix : EST.1.
Francais /2096/ .FRC97
https://www.bible.com/fr/bible/2096/EST.1.FRC97
chapterPrefix : EST.1.
{'EST.1.1': 'L’histoire rapportée ici est arrivée au temps du roi Xerxès , celui qui régna sur cent vingt-sept provinces, de l’Inde jusqu’à l’Éthiopie. ', 'EST.1.2+EST.1.3': 'Au cours de la troisième année de son règne, le roi, installé dans son palais royal de la citadelle de Suse, organisa un banquet pour tous ses hauts fonctionnaires et ses ministres ; il réunit auprès de lui les officiers de l’armée de Perse et de Médie, les gouverneurs et les chefs des provinces . ', 'EST.1.4': 'Il leur montra les richesses magnifiques et le luxe éclatant qui faisaient la gloire de son règne. Les festivités durèrent très longtemps, six mois au total. ', 'EST.1.5': 'A la fin de cette période, le roi offrit également un banquet aux gens de toutes conditions, riches ou pauvres, qui se trouvaient dans la citadelle de Suse. Cette fête se dé

In [0]:
! cp /content/csvs/EST.csv /content/drive/"My Drive"/African_Translator/datasets/YourVersion/csvs
! cp /content/csvs/DAN.csv /content/drive/"My Drive"/African_Translator/datasets/YourVersion/csvs

**ACT : pas de probleme, forcement la connexion était fini lors du scraping**

In [0]:
prefix_at = {'Francais':('/504/','.PDV2017'), 
             "Anglais":('/1/','.NIV'),  "BIBALDA_TA_PELDETTA":('/557/','.MASSANA'), 
             'Bulu':("/856/",".BULU09"),  'Guiziga':('/550/','.BEGDC'), "Fulfulde_Adamaoua":("/906/",'.FB'),  "Fulfulde_DC":('/905/','.FBDC'), 
              'KALATA_KO_SC_Gbaya':('/871/','.GB11'), 'KALATA_KO_DC_Gbaya':('/868/','.GB11DC'), 'Kapsiki_DC':('/1515/','.KBDC'),
              'Tupurri':('/892/','.TUPB')}
prefix_nt = {'Bafia':('/876/',".BAFNT"), 'Dii':('/1511/','.DIINT'), 'Ejagham':('/889/','.ETUNT'), 'Ghomala':('/907/','.GNT02'),  
             'Vute':('/887/','.NTV'), 'Limbum':('/1240/','.LNT'), 'MKPAMAN_AMVOE_Ewondo':('/1854/','.NTE12'), 'Mofa':('/908/',".MAFNT"), 
             "Mofu_Gudur":('/551/',".MOF"), "Ngiemboon":('/229/','.NBGM'), 'Doyayo':('/880/','.NTD'), "Guidar":('/1120/',".GDRNT85"),
            'Peere_Nt&Psalms':('/1237/','.PERE'), 'Samba_Leko':('/1118/','.SLNT'), "Du_na_sdik_na_wiini_Alaw":('/1855/',".MUSGUM")}
prefix_nt.update(prefix_at) # prefix_nt = {**prefix_at, **prefix_nt}

YourVersion_at = {}
YourVersion_nt = {'ACT.':28}

scrapperYv = ScrapperYv(main_link, prefix_at, prefix_nt, YourVersion_at, YourVersion_nt, prefix_sufix_reference=['/504/','.PDV2017'])
scrapperYv.meta_data(main_path = main_path, txt_separator="__SEPARATOR__", cell_error='__Error__', into_link="INTRO1")
scrapperYv.scrap(YourVersion_dic = scrapperYv.YourVersion_nt, prefix_dic = scrapperYv.prefix_nt)

https://www.bible.com/fr/bible/504/ACT.1.PDV2017
chapterPrefix : ACT.1.
Bafia /876/ .BAFNT
https://www.bible.com/fr/bible/876/ACT.1.BAFNT
chapterPrefix : ACT.1.
{'ACT.1.1': 'Tyoofil, ɓi kaàtà wɛ̀m à ǹtonto, ǹkalìghà ɓìɓan ɓìcèm ɓi Yeesus à Kìghà, rì ɓi à rèsàkà, à tumɛ̀n a rìtumɛ̀n ', 'ACT.1.2': 'à kpaŋ ŋwos anɛ à ɓaa lɛ, à mɛ reɓsag ɓiɓan ɓii rì ìtèɗ wu Kitɛntɛ̀ɛ̀n ki Nghay rì ɓɛ̀kɛ̀nì-fìtom ɓɛ̀ à ɓɔ̀ngà, à ɓɔg min aɗyoo.', 'ACT.1.3': 'Ajɛ̀m ì rìwu ɗii, à rèsɛ̀ngà ɓɔ, à ki rèsi ɓɔ ɓìrèmzɛnà lɛɛ̀gwey ɓi lɛ ɓɛɛ yi lɛ à rì cem; à rèsɛ̀ngà a lɔ̀ŋkaa, mɛ̀ru mɛ̀tin mɛɓɛɛ̀, à kakàlì ɓɔ ɓìɓan ɓi ɓi seesee kìlɔ̀ŋ ki Bɛ̀ll.', 'ACT.1.4': 'Ŋwos adɛŋ ànɛ Yeesus à ɗiɗìghà ǹɗaŋ ɗi ɓɛ̀yilàg ɓii, nni à siŋdɛ̀ngà ɓɔ lɛ: Kɛ̀ ɓi Pɛ̀sɛn a Yorisalɛm, ɓoŋina lɔŋ ǹkàg wu Tààta à kàkà mìni, wɛɛ̀wu ǹwogsìghà mìni; ', 'ACT.1.5': 'lɛ Zyaŋ à ɓàɓtìghà ɓum ɗì mɛ̀nig, kìɓan kifog mìnii ri ɓiì ɓàɓtɛɛ ri Kìtɛntɛ̀ɛ̀n ki Nghay di ɓɔɔ mɛru 

In [0]:
! cp /content/csvs/ACT.csv /content/drive/"My Drive"/African_Translator/datasets/YourVersion/csvs